In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


In [77]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

In [78]:
# consolidating income col values
y[y['income'] == '<=50K.'] = '<=50K'
y[y['income'] == '>50K.'] = '>50K'
y['income'] = y['income'].apply(lambda x: 1 if x == '<=50K' else 0)

/var/folders/t0/08jdm81x5hl2xjtdyy2hjzmh0000gn/T/ipykernel_44656/3281958407.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y['income'] == '<=50K.'] = '<=50K'
/var/folders/t0/08jdm81x5hl2xjtdyy2hjzmh0000gn/T/ipykernel_44656/3281958407.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y['income'] == '<=50K.'] = '<=50K'
/var/folders/t0/08jdm81x5hl2xjtdyy2hjzmh0000gn/T/ipykernel_44656/3281958407.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

#### Missing Values

In [79]:
# finds all rows that has a "?" or missing values, then filters dataframe to get rows that do not have those values
cols = X.columns
has = ~(X['age'] == X['age'])
for col in cols:
    has = ((has) | (X[col] =='?') | (X[col].isna()))

X = X[~has]
y = y[~has]

In [84]:
has.mean()
# since the percentage of rows with missing values is relatively low, 
# will procced with training and testing model with only rows with complete data

0.07411653904426518

#### Model Pipeline

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [43]:
categorical = ["race", "sex", "workclass", "education", "marital-status", "occupation", "relationship", "native-country"]
preproc = ColumnTransformer(
    transformers = [
        ("categorical", OneHotEncoder(handle_unknown='ignore', sparse_output = False), categorical),
        ("numerical", "passthrough", ["age", "hours-per-week", "fnlwgt", "capital-gain", "capital-loss"])
    ],
    remainder = "drop"
)

In [45]:
pl = Pipeline([
    ("preproc", preproc),
    ("model", LogisticRegression())
])
pl.fit(X_train, y_train)

/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['race', 'sex', 'workclass',
                                                   'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'native-country']),
                                                 ('numerical', 'passthrough',
                                                  ['age', 'hours-per-week',
                                                   'fnlwgt', 'capital-gain',
                                                   'capital-loss'])])),
                ('model', LogisticRegression())])

In [70]:
predicted = pl.predict(X_test)

In [71]:
# accuracy
(pd.Series(predicted) == y_test.reset_index()['income']).mean()


0.7924375322473649

In [72]:
# recall
recall_score(y_test, predicted)

0.9637986853723143

In [75]:
f1_score(y_test, predicted)

0.8746438746438746